In [1]:
from pymongo import MongoClient
import re

mc = MongoClient('192.168.194.171', port=27017)
db = mc.db_restT

In [2]:
tweets = db.restT.find(
    {
        "retweeted_status.lang" : "en",
        "retweeted_status.user.id_str" : "1130534810"
    }
).limit(1)
for tweet in tweets:
    print tweet['user']['id_str']
    print tweet['retweeted_status']['user']['id_str']
    print tweet['retweeted_status']['user']['name']
    print tweet['retweeted_status']['user']['screen_name']
    print tweet['retweeted_status']['retweet_count']
    print tweet['retweeted_status']['id_str']
    print tweet['retweeted_status']['text']
    print '---------------------------------------------------------------'

201871438
1130534810
NBA On Def Pen
NBAOnDefPen
8
611345721980256256
The Curry's #NBAFinals2015 http://t.co/ZVmvQI7zdD
---------------------------------------------------------------


In [3]:
retweets = {}
tweets = db.restT.find(
    {
        "retweeted_status.lang" : "en",
    }
).limit(1)
for tweet in tweets:
    retweets['author_id'] = tweet['retweeted_status']['user']['id_str']
    retweets['text']      = tweet['retweeted_status']['text']
    retweets['id_str']    = tweet['retweeted_status']['id_str']
    retweets['retweeter'] = {'user' : tweet['user']['id_str']}
    print str(retweets)
#    db.retweets.insert(retweet)

{'text': u'#LeBronsDick live at the #NBAFinals2015 \U0001f602 http://t.co/Uy4QSUMwfJ', 'author_id': u'228875931', 'retweeter': {'user': u'3042489974'}, 'id_str': u'609203651282882560'}


In [95]:
retweets = db.retweet.find()
for retweet in retweets:
    print retweet

{u'contributors': None, u'truncated': False, u'text': u'#LeBronsDick live at the #NBAFinals2015 \U0001f602 http://t.co/Uy4QSUMwfJ', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 609203651282882560L, u'favorite_count': 88, u'source': u'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', u'retweeted': False, u'coordinates': None, u'entities': {u'symbols': [], u'user_mentions': [], u'hashtags': [{u'indices': [0, 12], u'text': u'LeBronsDick'}, {u'indices': [25, 39], u'text': u'NBAFinals2015'}], u'urls': [], u'media': [{u'expanded_url': u'http://twitter.com/FashionistaKika/status/609203651282882560/photo/1', u'display_url': u'pic.twitter.com/Uy4QSUMwfJ', u'url': u'http://t.co/Uy4QSUMwfJ', u'media_url_https': u'https://pbs.twimg.com/media/CHRTfLmUAAAWwrr.jpg', u'id_str': u'609203644756393984', u'sizes': {u'small': {u'h': 396, u'w': 340, u'resize': u'fit'}, u'large': {u'h': 670, u'w': 574, u'resize': u'fit'}, u'medium': {u'h': 670, u'w': 57

In [53]:
db.restT.create_index("retweeted_status.id_str")
stuff = db.restT.aggregate( 
    [
        { "$match" : { "retweeted_status.lang" : "en"}},
        { "$group" : { "_id" : "$retweeted_status.id_str",
                       "userid" : {"$first" : "$retweeted_status.user.id_str"},
                       "name": { "$first" : "$retweeted_status.user.name"},
                       "screen_name" : {"$first" : "$retweeted_status.user.screen_name"},
                       "location" : { "$first" : "retweeted_status.user.location"},
                       "text" : { "$first" : "retweeted_status.text"},
                       "count" : {"$sum" : 1}}},
        { "$sort" :  { "count" : -1}},
        { "$project" : { "count" : 1,
                         "_id" : 1,
                         "name" : 1,
                         "screen_name" : 1,
                         "userid" : 1}},
        { "$limit" : 30}
    ])

In [54]:
for things in stuff:
    print things


{u'count': 27705, u'_id': u'611020181310025729', u'userid': u'26270913', u'name': u'Golden St. Warriors', u'screen_name': u'warriors'}
{u'count': 3607, u'_id': u'611021976379895808', u'userid': u'26270913', u'name': u'Golden St. Warriors', u'screen_name': u'warriors'}
{u'count': 1459, u'_id': u'610984655110696962', u'userid': u'26270913', u'name': u'Golden St. Warriors', u'screen_name': u'warriors'}
{u'count': 1248, u'_id': u'609182946881339392', u'userid': u'26270913', u'name': u'Golden St. Warriors', u'screen_name': u'warriors'}
{u'count': 1172, u'_id': u'609162872892096512', u'userid': u'26270913', u'name': u'Golden St. Warriors', u'screen_name': u'warriors'}
{u'count': 1124, u'_id': u'610651339014582274', u'userid': u'26270913', u'name': u'Golden St. Warriors', u'screen_name': u'warriors'}
{u'count': 1080, u'_id': u'610217943243190272', u'userid': u'2238576858', u'name': u'Ridiculousness', u'screen_name': u'WeLoveRobDyrdek'}
{u'count': 939, u'_id': u'611053800892207104', u'userid':

In [49]:
db.restT.create_index("retweeted_status.id_str")
stuff = db.restT.aggregate( 
    [
        { "$match" : { "retweeted_status.lang" : "en"}},
        { "$group" : { "_id" : "$retweeted_status.id_str",
                       "count" : {"$sum" : 1}}},
        { "$sort" :  { "count" : -1}},
        { "$project" : { "count" : 1,
                         "_id" : 1,
                         "name" : 1,
                         "screen_name" : 1}},
        { "$limit" : 30}
    ])

In [50]:
print type(stuff)
for things in stuff:
    print things

<class 'pymongo.command_cursor.CommandCursor'>
{u'count': 27705, u'_id': u'611020181310025729'}
{u'count': 3607, u'_id': u'611021976379895808'}
{u'count': 1459, u'_id': u'610984655110696962'}
{u'count': 1248, u'_id': u'609182946881339392'}
{u'count': 1172, u'_id': u'609162872892096512'}
{u'count': 1124, u'_id': u'610651339014582274'}
{u'count': 1080, u'_id': u'610217943243190272'}
{u'count': 939, u'_id': u'611053800892207104'}
{u'count': 900, u'_id': u'609202168613089280'}
{u'count': 872, u'_id': u'609172508521201665'}
{u'count': 856, u'_id': u'611021060939476992'}
{u'count': 808, u'_id': u'609204224459673600'}
{u'count': 785, u'_id': u'611029112187187200'}
{u'count': 751, u'_id': u'611024277274365952'}
{u'count': 730, u'_id': u'606514708561199104'}
{u'count': 724, u'_id': u'610995278750027777'}
{u'count': 692, u'_id': u'611020643488919552'}
{u'count': 682, u'_id': u'611028459586981888'}
{u'count': 675, u'_id': u'611020185818918912'}
{u'count': 634, u'_id': u'611108479462449152'}
{u'co